In [ ]:
# ============================================================================
# ИМПОРТ БИБЛИОТЕК
# ============================================================================

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from dateutil.relativedelta import relativedelta
import os
import warnings
from collections import defaultdict, deque
import logging
import pickle
import json
from pathlib import Path

from catboost import CatBoostRegressor, Pool
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Настройка логирования
logging.basicConfig(
    level=logging.INFO, 
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Настройка отображения pandas
pd.set_option("display.float_format", "{:,.2f}".format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Подавление предупреждений
warnings.filterwarnings('ignore')

# Настройка визуализации
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)

logger.info("Библиотеки успешно загружены")

In [ ]:
# ============================================================================
# КОНФИГУРАЦИЯ ПАРАМЕТРОВ
# ============================================================================

class ModelConfig:
    
    # Пути к данным
    DATA_DIR = Path("data")
    TRAIN_PATH = DATA_DIR / "train_data.parquet"
    
    # Директория для моделей
    MODEL_DIR = Path("models")
    MODEL_VERSION = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Временные границы разделения данных
    TRAIN_CUTOFF = "2025-01-31"       # Конец обучающей выборки
    VALIDATION_CUTOFF = "2025-03-31"  # Конец валидационной выборки
    # OOT (Out-of-Time) тестовая выборка: после VALIDATION_CUTOFF
    
    # Прогнозирование
    FORECAST_START = "2025-10-31"
    HORIZON_MONTHS = 6
    MIN_SAMPLES_PER_SEGMENT = 1000
    
    # Признаки модели
    CATEGORICAL_FEATURES = [
        'QUALITY_CODE',     
        'SUBJECT_KIND_ID',   
        'EC_SECTOR_ID'       
    ]
    
    BASE_FEATURES = [
        # Текущие и лаговые значения маржи
        'MARGIN', 'MARGIN_LAG1', 'MARGIN_LAG2', 'MARGIN_LAG3',
        # Скользящие средние
        'MARGIN_AVG_1M_LAG', 'MARGIN_AVG_2M_LAG', 'MARGIN_AVG_3M_LAG',
        'MARGIN_AVG_6M_LAG', 'MARGIN_AVG_12M_LAG',
        # Волатильность и тренд
        'MARGIN_STDDEV_12M_LAG', 'MARGIN_GROWTH_RATE_3M',
        # Временные признаки
        'MONTH_OF_YEAR', 'QUARTER_OF_YEAR', 'TENURE_MONTHS'
    ]
    
    # Маппинг сегментов клиентов
    SEGMENT_MAPPING = {
        '1026': 'small',              # МИКРО бизнес
        '1027': 'small',              # МАЛЫЙ бизнес
        '1022': 'large_and_middle',   # СРЕДНИЙ бизнес
        '1023': 'large_and_middle',   # КРУПНЫЙ бизнес
        '1040': 'large_and_middle',   # Прочие крупные
    }
    
    # Гиперпараметры CatBoost
    CATBOOST_PARAMS = {
        'iterations': 1600,           
        'depth': 4,                  
        'learning_rate': 0.05,        
        'l2_leaf_reg': 3,             
        'random_seed': 42,           
        'loss_function': 'RMSE',     
        'verbose': 100,               
        'early_stopping_rounds': 50   
    }
    
    @classmethod
    def ensure_directories(cls):
        cls.MODEL_DIR.mkdir(parents=True, exist_ok=True)
        (cls.MODEL_DIR / cls.MODEL_VERSION).mkdir(parents=True, exist_ok=True)

# Инициализация конфигурации
ModelConfig.ensure_directories()

logger.info(f"Версия модели: {ModelConfig.MODEL_VERSION}")
logger.info(f"Обучающая выборка: до {ModelConfig.TRAIN_CUTOFF}")
logger.info(f"Валидационная выборка: {ModelConfig.TRAIN_CUTOFF} - {ModelConfig.VALIDATION_CUTOFF}")
logger.info(f"OOT тестовая выборка: после {ModelConfig.VALIDATION_CUTOFF}")

In [ ]:
# ============================================================================
# ВСПОМОГАТЕЛЬНЫЕ ФУНКЦИИ
# ============================================================================

def load_parquet_data(file_path: Path) -> pd.DataFrame:
    if not file_path.exists():
        logger.warning(f"Файл не найден: {file_path}")
        return pd.DataFrame()
    
    try:
        df = pd.read_parquet(file_path)
        logger.info(f"Загружено {len(df):,} записей из {file_path.name}")
        return df
    except Exception as e:
        logger.error(f"Ошибка загрузки {file_path}: {e}")
        return pd.DataFrame()


def preprocess_categorical_features(df: pd.DataFrame, 
                                   categorical_cols: list) -> pd.DataFrame:
    df_processed = df.copy()
    
    for col in categorical_cols:
        if col in df_processed.columns:
            # Заполнение пропусков
            df_processed[col] = df_processed[col].fillna('UNKNOWN')
            # Приведение к строке
            df_processed[col] = df_processed[col].astype(str)
            # Удаление .0 из строковых представлений чисел
            df_processed[col] = df_processed[col].str.replace('.0', '', regex=False)
    
    return df_processed


def transform_target(y: pd.Series) -> pd.Series:
    return np.sign(y) * np.log1p(np.abs(y))


logger.info("Вспомогательные функции загружены")

In [ ]:
# ============================================================================
# ЗАГРУЗКА И ПРЕДОБРАБОТКА ДАННЫХ
# ============================================================================

logger.info("Начало загрузки данных...")

# Загрузка данных из Parquet
df_train = load_parquet_data(ModelConfig.TRAIN_PATH)

# Проверка успешности загрузки
if df_train.empty:
    logger.error("Ошибка: данные не загружены!")
    logger.info("Надо прогнать 'data_loader.ipynb'")
else:
    logger.info(f"Обучающая выборка: {len(df_train):,} записей")
    logger.info(f"Уникальных клиентов: {df_train['CLIENT_ID'].nunique():,}")

In [ ]:
# ============================================================================
# EXPLORATORY DATA ANALYSIS
# ============================================================================

logger.info("Начало разведочного анализа данных...")

print("\n" + "="*80)
print("ОБЩАЯ ИНФОРМАЦИЯ О ДАННЫХ")
print("="*80)
print(f"Размер данных: {df_train.shape[0]:,} строк x {df_train.shape[1]} столбцов")
print(f"Период данных: {df_train['MONTH_END'].min()} - {df_train['MONTH_END'].max()}")
print(f"Уникальных клиентов: {df_train['CLIENT_ID'].nunique():,}")

print("\n" + "="*80)
print("ПРОПУЩЕННЫЕ ЗНАЧЕНИЯ")
print("="*80)
missing = df_train.isnull().sum()
missing_pct = 100 * missing / len(df_train)
missing_df = pd.DataFrame({
    'Столбец': missing.index,
    'Пропусков': missing.values,
    'Процент': missing_pct.values
})
missing_df = missing_df[missing_df['Пропусков'] > 0].sort_values('Пропусков', ascending=False)
if len(missing_df) > 0:
    print(missing_df.to_string(index=False))
else:
    print("Пропущенных значений не обнаружено")

print("\n" + "="*80)
print("РАСПРЕДЕЛЕНИЕ ЦЕЛЕВОЙ ПЕРЕМЕННОЙ (TARGET_NEXT_MARGIN)")
print("="*80)
target_stats = df_train['TARGET_NEXT_MARGIN'].describe()
print(target_stats)

print("\n" + "="*80)
print("РАСПРЕДЕЛЕНИЕ ПО СЕГМЕНТАМ")
print("="*80)
segment_counts = df_train['SEGMENT_ID'].value_counts()
for segment, count in segment_counts.items():
    pct = 100 * count / len(df_train)
    print(f"{segment}: {count:,} ({pct:.1f}%)")

print("\n" + "="*80)
print("СТАТИСТИКА МАРЖИ ПО СЕГМЕНТАМ")
print("="*80)
margin_by_segment = df_train.groupby('SEGMENT_ID')['TARGET_NEXT_MARGIN'].agg([
    ('count', 'count'),
    ('mean', 'mean'),
    ('median', 'median'),
    ('std', 'std'),
    ('min', 'min'),
    ('max', 'max')
])
print(margin_by_segment)

# Визуализация распределения целевой переменной
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Гистограмма
axes[0].hist(df_train['TARGET_NEXT_MARGIN'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('TARGET_NEXT_MARGIN')
axes[0].set_ylabel('Частота')
axes[0].set_title('Распределение целевой переменной')
axes[0].grid(alpha=0.3)

# Box plot по сегментам
df_train.boxplot(column='TARGET_NEXT_MARGIN', by='SEGMENT_ID', ax=axes[1])
axes[1].set_xlabel('Сегмент')
axes[1].set_ylabel('TARGET_NEXT_MARGIN')
axes[1].set_title('Распределение маржи по сегментам')
plt.suptitle('')

plt.tight_layout()
plt.show()

In [ ]:
# ============================================================================
# АГРЕГАЦИЯ СЕГМЕНТОВ
# ============================================================================

logger.info("Применение маппинга сегментов...")

print("\n" + "="*80)
print("ИСХОДНАЯ СЕГМЕНТАЦИЯ КЛИЕНТОВ")
print("="*80)
print(df_train['SEGMENT_ID'].value_counts().sort_index())

# Применение маппинга сегментов
df_train['SEGMENT_ID'] = (
    df_train['SEGMENT_ID']
    .astype(str)
    .map(ModelConfig.SEGMENT_MAPPING)
)

# Удаление строк с неизвестными сегментами
rows_before = len(df_train)
df_train = df_train.dropna(subset=['SEGMENT_ID'])
rows_after = len(df_train)
rows_removed = rows_before - rows_after

if rows_removed > 0:
    logger.info(f"Удалено {rows_removed:,} записей с неизвестными сегментами")

print("\n" + "="*80)
print("УКРУПНЕННАЯ СЕГМЕНТАЦИЯ (2 СЕГМЕНТА)")
print("="*80)
print(df_train['SEGMENT_ID'].value_counts().sort_index())

# Детальная статистика по сегментам
print("\n" + "="*80)
print("СТАТИСТИКА ПО СЕГМЕНТАМ")
print("="*80)

for segment_name in sorted(df_train['SEGMENT_ID'].unique()):
    segment_data = df_train[df_train['SEGMENT_ID'] == segment_name]
    
    n_records = len(segment_data)
    n_clients = segment_data['CLIENT_ID'].nunique()
    pct_records = 100 * n_records / len(df_train)
    avg_margin = segment_data['TARGET_NEXT_MARGIN'].mean()
    
    print(f"\n{segment_name.upper()}:")
    print(f"  Записей: {n_records:>12,} ({pct_records:>5.1f}%)")
    print(f"  Клиентов: {n_clients:>11,}")

logger.info("Сегментация применена")

In [ ]:
# ============================================================================
# ФОРМИРОВАНИЕ ПРИЗНАКОВОГО ПРОСТРАНСТВА
# ============================================================================

logger.info("Подготовка признаков...")

# Все категориальные признаки (включая SEGMENT_ID)
all_categorical_features = ModelConfig.CATEGORICAL_FEATURES + ['SEGMENT_ID']

# Предобработка категориальных признаков
df_train_processed = preprocess_categorical_features(
    df_train, all_categorical_features
)

# Полный список признаков
all_features = (
    ['SEGMENT_ID'] + 
    ModelConfig.BASE_FEATURES + 
    ModelConfig.CATEGORICAL_FEATURES
)

# Фильтрация доступных признаков
available_features = [
    feature for feature in all_features 
    if feature in df_train_processed.columns
]

# Заполнение пропусков в числовых признаках
numeric_features = [
    feature for feature in available_features 
    if feature not in all_categorical_features
]

for feature in numeric_features:
    df_train_processed[feature] = df_train_processed[feature].fillna(0.0)

logger.info(f"Всего признаков: {len(available_features)}")
logger.info(f"  Числовые: {len(numeric_features)}")
logger.info(f"  Категориальные: {len(ModelConfig.CATEGORICAL_FEATURES)}")

print(f"\nСписок признаков модели:")
print(f"  Сегментация: SEGMENT_ID")
print(f"  Числовые: {', '.join(numeric_features[:5])}...")
print(f"  Категориальные: {', '.join(ModelConfig.CATEGORICAL_FEATURES)}")

In [ ]:
# ============================================================================
# КЛАСС СЕГМЕНТИРОВАННОЙ МОДЕЛИ CLTV
# ============================================================================

class SegmentedCLTVModel:
    
    def __init__(self, model_params: dict):
        self.models = {}
        self.metrics = {}
        self.feature_importance = {}
        self.model_params = model_params
        
    def train_segment_model(
        self,
        segment_id: str,
        X_train: pd.DataFrame,
        y_train: pd.Series,
        X_val: pd.DataFrame,
        y_val: pd.Series,
        X_test: pd.DataFrame,
        y_test: pd.Series,
        categorical_features: list = None
    ) -> tuple:
        logger.info(f"\nОбучение сегмента: {segment_id}")
        logger.info(f"  Train: {len(X_train):,} записей")
        logger.info(f"  Validation: {len(X_val):,} записей")
        logger.info(f"  OOT Test: {len(X_test):,} записей")
        
        # Определение индексов категориальных признаков
        categorical_indices = []
        if categorical_features:
            feature_names = X_train.columns.tolist()
            categorical_indices = [
                feature_names.index(feature) 
                for feature in categorical_features 
                if feature in feature_names
            ]
        
        # Создание Pool объектов для CatBoost
        train_pool = Pool(X_train, y_train, cat_features=categorical_indices)
        val_pool = Pool(X_val, y_val, cat_features=categorical_indices)
        test_pool = Pool(X_test, y_test, cat_features=categorical_indices)
        
        # Обучение модели
        model = CatBoostRegressor(**self.model_params)
        model.fit(
            train_pool, 
            eval_set=val_pool, 
            use_best_model=True
        )
        
        # Расчет метрик на всех выборках
        predictions = {
            'train': model.predict(X_train),
            'val': model.predict(X_val),
            'test': model.predict(X_test)
        }
        
        metrics = {
            # Обучающая выборка
            'train_r2': r2_score(y_train, predictions['train']),
            'train_rmse': np.sqrt(mean_squared_error(y_train, predictions['train'])),
            'train_mae': mean_absolute_error(y_train, predictions['train']),
            'train_samples': len(X_train),
            # Валидационная выборка
            'val_r2': r2_score(y_val, predictions['val']),
            'val_rmse': np.sqrt(mean_squared_error(y_val, predictions['val'])),
            'val_mae': mean_absolute_error(y_val, predictions['val']),
            'val_samples': len(X_val),
            # OOT тестовая выборка
            'test_r2': r2_score(y_test, predictions['test']),
            'test_rmse': np.sqrt(mean_squared_error(y_test, predictions['test'])),
            'test_mae': mean_absolute_error(y_test, predictions['test']),
            'test_samples': len(X_test),
        }
        
        # Feature importance
        importance_df = pd.DataFrame({
            'feature': X_train.columns,
            'importance': model.feature_importances_
        }).sort_values('importance', ascending=False)
        
        # Вывод результатов
        logger.info("Результаты обучения:")
        logger.info(f"  Train R²: {metrics['train_r2']:.4f}")
        logger.info(f"  Val R²:   {metrics['val_r2']:.4f}")
        logger.info(f"  Test R²:  {metrics['test_r2']:.4f} (OOT)")
        
        # Сохранение результатов
        self.models[segment_id] = model
        self.metrics[segment_id] = metrics
        self.feature_importance[segment_id] = importance_df
        
        return model, metrics
    
    def predict(self, segment_id: str, X: pd.DataFrame) -> np.ndarray:
        if segment_id not in self.models:
            raise ValueError(f"Модель для сегмента '{segment_id}' не обучена")
        
        # Удаление SEGMENT_ID из признаков если присутствует
        X_pred = X.drop('SEGMENT_ID', axis=1) if 'SEGMENT_ID' in X.columns else X
        
        return self.models[segment_id].predict(X_pred)


logger.info("Класс SegmentedCLTVModel определен")

In [ ]:
# ============================================================================
# ПОДГОТОВКА ОБУЧАЮЩИХ ДАННЫХ
# Разделение на Train / Validation / OOT Test
# ============================================================================

logger.info("Формирование обучающих выборок...")

# Стабилизирующая трансформация целевой переменной
df_train_processed['target_transformed'] = transform_target(
    df_train_processed['TARGET_NEXT_MARGIN']
)

# Временное разделение данных
train_end_date = pd.to_datetime(ModelConfig.TRAIN_CUTOFF)
val_end_date = pd.to_datetime(ModelConfig.VALIDATION_CUTOFF)

month_end_dates = pd.to_datetime(df_train_processed['MONTH_END'])

train_mask = month_end_dates <= train_end_date
val_mask = (month_end_dates > train_end_date) & (month_end_dates <= val_end_date)
test_mask = month_end_dates > val_end_date

# Статистика разделения
print("\n" + "="*80)
print("РАЗДЕЛЕНИЕ ДАННЫХ")
print("="*80)
print(f"Train:      {train_mask.sum():>10,} записей ({100*train_mask.sum()/len(df_train_processed):>5.1f}%)")
print(f"Validation: {val_mask.sum():>10,} записей ({100*val_mask.sum()/len(df_train_processed):>5.1f}%)")
print(f"OOT Test:   {test_mask.sum():>10,} записей ({100*test_mask.sum()/len(df_train_processed):>5.1f}%)")

# Подготовка данных по сегментам
segment_datasets = {}

for segment_id in sorted(df_train_processed['SEGMENT_ID'].unique()):
    segment_mask = df_train_processed['SEGMENT_ID'] == segment_id
    
    # Комбинированные маски для каждой выборки
    train_segment_mask = segment_mask & train_mask
    val_segment_mask = segment_mask & val_mask
    test_segment_mask = segment_mask & test_mask
    
    # Признаки без SEGMENT_ID
    model_features = [f for f in available_features if f != 'SEGMENT_ID']
    
    # Формирование наборов данных
    segment_datasets[segment_id] = {
        'X_train': df_train_processed.loc[train_segment_mask, model_features],
        'y_train': df_train_processed.loc[train_segment_mask, 'target_transformed'],
        'X_val': df_train_processed.loc[val_segment_mask, model_features],
        'y_val': df_train_processed.loc[val_segment_mask, 'target_transformed'],
        'X_test': df_train_processed.loc[test_segment_mask, model_features],
        'y_test': df_train_processed.loc[test_segment_mask, 'target_transformed']
    }
    
    print(f"\n{segment_id.upper()}:")
    print(f"  Train: {len(segment_datasets[segment_id]['X_train']):>10,} записей")
    print(f"  Val:   {len(segment_datasets[segment_id]['X_val']):>10,} записей")
    print(f"  Test:  {len(segment_datasets[segment_id]['X_test']):>10,} записей")

logger.info("Данные подготовлены для обучения")

In [ ]:
# ============================================================================
# ОБУЧЕНИЕ МОДЕЛЕЙ
# ============================================================================

logger.info("\nНачало обучения моделей...")

print("\n" + "="*80)
print("ОБУЧЕНИЕ СЕГМЕНТИРОВАННЫХ МОДЕЛЕЙ CLTV")
print("="*80)

# Инициализация модели
cltv_model = SegmentedCLTVModel(model_params=ModelConfig.CATBOOST_PARAMS)

# Обучение моделей для каждого сегмента
for segment_id in sorted(segment_datasets.keys()):
    print(f"\n{'='*80}")
    print(f"СЕГМЕНТ: {segment_id.upper()}")
    print(f"{'='*80}")
    
    dataset = segment_datasets[segment_id]
    
    model, metrics = cltv_model.train_segment_model(
        segment_id=segment_id,
        X_train=dataset['X_train'],
        y_train=dataset['y_train'],
        X_val=dataset['X_val'],
        y_val=dataset['y_val'],
        X_test=dataset['X_test'],
        y_test=dataset['y_test'],
        categorical_features=ModelConfig.CATEGORICAL_FEATURES
    )
    
    # Вывод топ-10 важных признаков
    print(f"\nТоп-10 важных признаков:")
    importance_df = cltv_model.feature_importance[segment_id]
    for idx, row in importance_df.head(10).iterrows():
        print(f"  {row['feature']:30s}: {row['importance']:6.2f}")

print("\n" + "="*80)
print("ОБУЧЕНИЕ ЗАВЕРШЕНО")
print("="*80)

logger.info("Все модели успешно обучены")

In [ ]:
# ============================================================================
# СВОДКА РЕЗУЛЬТАТОВ
# ============================================================================

print("\n" + "="*80)
print("ИТОГОВЫЕ МЕТРИКИ КАЧЕСТВА МОДЕЛЕЙ")
print("="*80)

# Формирование сводной таблицы
summary_records = []

for segment_id, metrics in cltv_model.metrics.items():
    summary_records.append({
        'Сегмент': segment_id,
        'Train_R²': f"{metrics['train_r2']:.4f}",
        'Val_R²': f"{metrics['val_r2']:.4f}",
        'Test_R²': f"{metrics['test_r2']:.4f}",
        'Train_RMSE': f"{metrics['train_rmse']:.2f}",
        'Val_RMSE': f"{metrics['val_rmse']:.2f}",
        'Test_RMSE': f"{metrics['test_rmse']:.2f}",
        'Train_MAE': f"{metrics['train_mae']:.2f}",
        'Val_MAE': f"{metrics['val_mae']:.2f}",
        'Test_MAE': f"{metrics['test_mae']:.2f}",
    })

summary_df = pd.DataFrame(summary_records)
print("\n" + summary_df.to_string(index=False))

# Расчет средних метрик
avg_metrics = {
    'train_r2': np.mean([m['train_r2'] for m in cltv_model.metrics.values()]),
    'val_r2': np.mean([m['val_r2'] for m in cltv_model.metrics.values()]),
    'test_r2': np.mean([m['test_r2'] for m in cltv_model.metrics.values()]),
    'test_rmse': np.mean([m['test_rmse'] for m in cltv_model.metrics.values()]),
    'test_mae': np.mean([m['test_mae'] for m in cltv_model.metrics.values()]),
}

print(f"\n{'='*80}")
print("СРЕДНИЕ МЕТРИКИ ПО ВСЕМ СЕГМЕНТАМ")
print(f"{'='*80}")
print(f"  Train R²:     {avg_metrics['train_r2']:.4f}")
print(f"  Val R²:       {avg_metrics['val_r2']:.4f}")
print(f"  Test R² (OOT): {avg_metrics['test_r2']:.4f}  <- Финальная метрика качества")
print(f"  Test RMSE:    {avg_metrics['test_rmse']:.2f}")
print(f"  Test MAE:     {avg_metrics['test_mae']:.2f}")
print(f"{'='*80}")

# Анализ стабильности модели
print(f"\nАНАЛИЗ СТАБИЛЬНОСТИ МОДЕЛЕЙ:")
print(f"{'='*80}")

for segment_id, metrics in cltv_model.metrics.items():
    train_val_diff = abs(metrics['train_r2'] - metrics['val_r2'])
    val_test_diff = abs(metrics['val_r2'] - metrics['test_r2'])
    
    print(f"\n{segment_id.upper()}:")
    print(f"  |Train R² - Val R²|:  {train_val_diff:.4f}")
    print(f"  |Val R² - Test R²|:   {val_test_diff:.4f}")
    
    if val_test_diff < 0.05:
        status = "Стабильная модель"
    elif val_test_diff < 0.10:
        status = "Умеренная деградация"
    else:
        status = "Сильная деградация"
    
    print(f"  Статус: {status}")

logger.info(f"Финальная OOT R²: {avg_metrics['test_r2']:.4f}")

In [ ]:
# ============================================================================
# СОХРАНЕНИЕ МОДЕЛЕЙ И АРТЕФАКТОВ
# ============================================================================

logger.info("Сохранение результатов...")

save_dir = ModelConfig.MODEL_DIR / ModelConfig.MODEL_VERSION

# Сохранение моделей CatBoost
for segment_id, model in cltv_model.models.items():
    model_path = save_dir / f"cltv_model_{segment_id}.cbm"
    model.save_model(str(model_path))
    logger.info(f"Сохранена модель: {model_path.name}")

# Сохранение сводки метрик
metrics_path = save_dir / "model_metrics.csv"
summary_df.to_csv(metrics_path, index=False)
logger.info(f"Сохранены метрики: {metrics_path.name}")

# Сохранение feature importance
for segment_id, importance_df in cltv_model.feature_importance.items():
    importance_path = save_dir / f"feature_importance_{segment_id}.csv"
    importance_df.to_csv(importance_path, index=False)
    logger.info(f"Сохранена важность признаков: {importance_path.name}")

# Сохранение метаданных
metadata = {
    'model_version': ModelConfig.MODEL_VERSION,
    'training_date': datetime.now().isoformat(),
    'segments': list(cltv_model.models.keys()),
    'features': available_features,
    'segment_mapping': ModelConfig.SEGMENT_MAPPING,
    'model_parameters': ModelConfig.CATBOOST_PARAMS,
    'data_split': {
        'train_cutoff': ModelConfig.TRAIN_CUTOFF,
        'validation_cutoff': ModelConfig.VALIDATION_CUTOFF,
        'train_size': int(train_mask.sum()),
        'val_size': int(val_mask.sum()),
        'test_size': int(test_mask.sum())
    },
    'performance_metrics': {
        seg_id: {
            k: float(v) if isinstance(v, (int, float, np.number)) else v
            for k, v in metrics.items()
        }
        for seg_id, metrics in cltv_model.metrics.items()
    },
    'average_metrics': avg_metrics
}

metadata_path = save_dir / "model_metadata.json"
with open(metadata_path, 'w', encoding='utf-8') as f:
    json.dump(metadata, f, indent=2, ensure_ascii=False)
logger.info(f"Сохранены метаданные: {metadata_path.name}")

# Сохранение объекта модели
model_object_path = save_dir / "cltv_model_object.pkl"
with open(model_object_path, 'wb') as f:
    pickle.dump(cltv_model, f)
logger.info(f"Сохранен объект модели: {model_object_path.name}")

print(f"\n{'='*80}")
print(f"РЕЗУЛЬТАТЫ СОХРАНЕНЫ")
print(f"{'='*80}")
print(f"Директория: {save_dir}")
print(f"\nСохраненные файлы:")
print(f"  - Модели CatBoost: {len(cltv_model.models)} файлов")
print(f"  - Метрики: model_metrics.csv")
print(f"  - Feature importance: {len(cltv_model.models)} файлов")
print(f"  - Метаданные: model_metadata.json")
print(f"  - Объект модели: cltv_model_object.pkl")
print(f"{'='*80}")

In [ ]:
# ============================================================================
# ВИЗУАЛИЗАЦИЯ РЕЗУЛЬТАТОВ
# ============================================================================

logger.info("Создание визуализаций...")

# График сравнения R² на разных выборках
fig, ax = plt.subplots(figsize=(14, 7))

segments = list(cltv_model.metrics.keys())
x_pos = np.arange(len(segments))
bar_width = 0.25

r2_train = [cltv_model.metrics[seg]['train_r2'] for seg in segments]
r2_val = [cltv_model.metrics[seg]['val_r2'] for seg in segments]
r2_test = [cltv_model.metrics[seg]['test_r2'] for seg in segments]

bars1 = ax.bar(x_pos - bar_width, r2_train, bar_width, 
               label='Train R²', alpha=0.8, color='#3498db')
bars2 = ax.bar(x_pos, r2_val, bar_width, 
               label='Validation R²', alpha=0.8, color='#2ecc71')
bars3 = ax.bar(x_pos + bar_width, r2_test, bar_width, 
               label='OOT Test R²', alpha=0.8, color='#e74c3c')

ax.set_xlabel('Сегмент', fontsize=12, fontweight='bold')
ax.set_ylabel('R² (коэффициент детерминации)', fontsize=12, fontweight='bold')
ax.set_title('Сравнение качества моделей на разных выборках', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_xticks(x_pos)
ax.set_xticklabels([seg.replace('_', ' ').title() for seg in segments])
ax.legend(loc='lower right', fontsize=11)
ax.grid(axis='y', alpha=0.3, linestyle='--')
ax.axhline(y=0.8, color='gray', linestyle='--', alpha=0.5, linewidth=1)

# Добавление значений на столбцы
for i, (t, v, o) in enumerate(zip(r2_train, r2_val, r2_test)):
    ax.text(i - bar_width, t + 0.01, f'{t:.3f}', 
            ha='center', va='bottom', fontsize=9)
    ax.text(i, v + 0.01, f'{v:.3f}', 
            ha='center', va='bottom', fontsize=9)
    ax.text(i + bar_width, o + 0.01, f'{o:.3f}', 
            ha='center', va='bottom', fontsize=9, fontweight='bold')

plt.tight_layout()
plot_path = save_dir / 'model_performance_comparison.png'
plt.savefig(plot_path, dpi=150, bbox_inches='tight')
logger.info(f"Сохранен график: {plot_path.name}")
plt.show()

print(f"\nВизуализация сохранена: {plot_path}")